In [1]:
!pip install optuna

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import optuna

In [2]:
train_data = pd.read_csv('data/train_prepared_oneliner_w_deltas_n_divs.csv')
test_data = pd.read_csv('data/test_prepared_oneliner_w_deltas_n_divs.csv')
train_data.head()

,P10_W0,P10_W1,P10_W2,P10_W3,P11_W0,P11_W1,P11_W2,P11_W3,P12_W0,P12_W1,...,P25_W3-2,P25_W3-0,P26_W1-0,P26_W2-1,P26_W3-2,P26_W3-0,P27_W1-0,P27_W2-1,P27_W3-2,P27_W3-0
0,0.614911,0.233876,0.760227,-0.062077,-1.841831,-0.237328,0.614045,-0.499621,-1.681824,-1.681824,...,-0.139155,0.323217,-0.399568,-1.078469,0.851133,-0.626903,0.202123,-0.837158,0.848628,0.213592
1,0.468424,0.614911,-0.243699,-0.039009,-0.025675,-0.436949,-0.658135,-0.114774,-1.381967,-0.779604,...,-0.131692,0.093221,0.635130,0.291939,1.012301,1.939370,0.565416,0.726801,-0.077080,1.215138
2,-0.889279,-0.216327,-0.520669,0.114520,-1.207994,-1.347325,-0.134329,-0.783241,1.141776,1.141776,...,0.583060,0.019954,-0.276790,0.361082,0.560520,0.644812,-0.222499,0.425951,-0.212453,-0.009001
3,0.047188,-0.243699,0.695793,-1.021933,-0.114774,0.038967,-0.556959,-0.063885,1.512694,1.512694,...,0.000000,0.000000,-0.153241,0.589407,0.293758,0.729925,0.053212,0.090980,0.200677,0.344869
4,-0.264865,-0.987172,-0.920027,-1.257932,-1.581778,-1.557080,-1.841831,-1.513413,0.504723,0.806938,...,-0.066926,0.030087,0.310174,0.252770,-0.732799,-0.169856,-0.307380,0.000000,0.000000,-0.307380


In [5]:
X_train = train_data
y_train = X_train['target']
X_train = X_train.drop(['target'], axis=1)
X_train.head()
X_test = test_data
X_test = X_test.set_index('Id')

In [6]:
y_train

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
7710    0.0
7711    0.0
7712    0.0
7713    0.0
7714    1.0
Name: target, Length: 7715, dtype: float64

In [7]:
train_data_with_random = X_train.copy()
train_data_with_random.insert(X_train.shape[1], "random", np.random.rand(X_train.shape[0]), True)
train_data_with_random['target'] = y_train

corr_matrix = train_data_with_random.corr()
corelations = corr_matrix["target"].sort_values(ascending=False)
unnececary_by_cors = list(corelations[corelations < corelations['random']].index)
unnececary_by_cors

['P6_W3-0',
 'P23_W3-2',
 'P18_W3-2',
 'P24_W2',
 'P2_W3-0',
 'P20_W3-2',
 'P2_W3-2',
 'P24_W1',
 'P1_W1/0',
 'P8_W2/1',
 'P23_W2-1',
 'P16_W3-0',
 'P21_W2',
 'P9_W3-2',
 'P27_W3-2',
 'P6_W1-0',
 'P5_W3/2',
 'P1_W3/2',
 'P8_W2-1',
 'P6_W3-2',
 'P7_W3-2',
 'P19_W1/0',
 'P3_W2',
 'P20_W2-1',
 'P16_W1-0',
 'P12_W2-1',
 'P21_W1-0',
 'P26_W1-0',
 'P1_W1-0',
 'P24_W2-1',
 'P14_W3-2',
 'P17_W3-0',
 'P22_W3-2',
 'P5_W1/0',
 'P14_W0',
 'P1_W3-2',
 'P26_W3/0',
 'P13_W3-0',
 'P14_W1',
 'P24_W0',
 'P1_W3-0',
 'P6_W2-1',
 'P26_W3/2',
 'P18_W1-0',
 'P17_W2-1',
 'P11_W1-0',
 'P14_W3',
 'P14_W1-0',
 'P4_W2-1',
 'P9_W2-1',
 'P22_W3-0',
 'V7',
 'P5_W1-0',
 'P26_W1/0',
 'P14_W2',
 'P14_W3-0',
 'P11_W3-2',
 'P1_W2-1',
 'P26_W2-1',
 'P11_W0',
 'P9_W3-0',
 'P4_W1-0',
 'P23_W1-0',
 'P7_W1-0',
 'P14_W2-1',
 'P11_W1',
 'P20_W3',
 'V5',
 'P23_W3/0',
 'P25_W1-0',
 'P19_W2-1',
 'P13_W2-1',
 'P25_W3-2',
 'P19_W3-2',
 'P21_W2-1',
 'P17_W1-0',
 'P21_W3',
 'P8_W1-0',
 'P13_W1-0',
 'P20_W2',
 'P9_W1-0',
 'P9_W1/0',
 '

In [9]:
from sklearn.ensemble import RandomForestClassifier

temp_X_train = train_data_with_random.drop(columns=["target"])

forest_cls = RandomForestClassifier(n_jobs=-1)
forest_cls.fit(temp_X_train, y_train)


forest_necessity = pd.Series(dict(sorted(zip(temp_X_train.columns, forest_cls.feature_importances_), 
                    reverse=True, 
                    key=lambda x: x[1])))
unnececary_by_forest = list(forest_necessity[forest_necessity < forest_necessity['random']].index)
unnececary_by_forest
# for name, importance in sorted(
#                     zip(temp_X_train.columns, forest_cls.feature_importances_), 
#                     reverse=True, 
#                     key=lambda x: x[1]):
#     print(f"{name} \t-> {importance}")

['P26_W2/1',
 'V18',
 'P26_W3-0',
 'P2_W3/0',
 'P25_W3/2',
 'P27_W3',
 'V21',
 'P18_W2',
 'P3_W3/0',
 'P4_W3',
 'P7_W1',
 'P18_W3',
 'P21_W3-0',
 'V10',
 'P19_W3/0',
 'P26_W3/2',
 'P17_W3/0',
 'P7_W3',
 'P24_W3-2',
 'P1_W3-2',
 'P18_W1',
 'P6_W2/1',
 'P1_W2-1',
 'P1_W3-0',
 'P15_W3/2',
 'P24_W3',
 'P21_W0',
 'P14_W2',
 'P7_W0',
 'P19_W1/0',
 'P15_W2/1',
 'P18_W1/0',
 'V8',
 'P19_W3-0',
 'P4_W0',
 'P13_W3',
 'P4_W2',
 'P27_W1',
 'P24_W3/0',
 'P22_W3/0',
 'P21_W2',
 'P21_W3/0',
 'P22_W1/0',
 'P15_W1/0',
 'P1_W2/1',
 'P19_W2-1',
 'P19_W1-0',
 'P19_W3-2',
 'P8_W2',
 'P14_W3',
 'P17_W3-0',
 'P22_W2/1',
 'P22_W2-1',
 'P17_W2/1',
 'P2_W3-2',
 'V11',
 'P2_W3/2',
 'P19_W2/1',
 'V16',
 'P4_W1',
 'P15_W2-1',
 'P18_W0',
 'V7',
 'P13_W0',
 'P6_W2-1',
 'P26_W2-1',
 'P22_W3/2',
 'P14_W0',
 'P5_W0',
 'P6_W3-0',
 'P12_W3/0',
 'P24_W3/2',
 'P26_W3/0',
 'P18_W3/0',
 'P14_W1',
 'P22_W3-0',
 'P24_W3-0',
 'P3_W1',
 'P2_W2/1',
 'P15_W1-0',
 'P6_W1/0',
 'V20',
 'P22_W3-2',
 'P24_W1-0',
 'P1_W3/0',
 'P3_W2',
 

In [10]:
COLS_TO_DROP = list(set(unnececary_by_forest) & set(unnececary_by_cors))

In [11]:
X_train = X_train.drop(columns=COLS_TO_DROP)
X_test = X_test.drop(columns=COLS_TO_DROP)

In [12]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [13]:
X_train = reduce_memory_usage(X_train)
X_test = reduce_memory_usage(X_test)

Mem. usage decreased to 3.69 Mb (75.0% reduction)
Mem. usage decreased to 1.61 Mb (74.7% reduction)


In [14]:
from sklearn.model_selection import StratifiedKFold

In [15]:
def objective(trial):

    param_grid = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
        "metric": "auc"
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
        X_train_1, X_val = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_1, y_val = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model = lgb.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train_1,
            y_train_1,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',
            early_stopping_rounds=100,
        )
        y_pred = model.predict_proba(X_val)[:, 1]
        cv_scores[idx] = roc_auc_score(y_val, y_pred)

    return np.mean(cv_scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2022-01-23 19:22:00,593] A new study created in memory with name: no-name-dba8feca-246c-4ad5-a5ca-bdc7816c38f4
C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCach

[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=7100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7100
[LightGBM] [Warning] min_gain_to_split is set=11.044175902385101, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.044175902385101
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current value: lambda_l2=15
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[2]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[3]	valid_0's binary_logloss: 0.583906	valid_0's auc

C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=7100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7100
[LightGBM] [Warning] min_gain_to_split is set=11.044175902385101, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.044175902385101
[LightGBM] [Warning] lambda_l1 is set=45, reg_alpha=0.0 will be ignored. Current value: lambda_l1=45
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current value: lambda_l2=15
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[2]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[3]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[4]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[5]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[6

C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2022-01-23 19:22:01,526] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 10000, 'learning_rate': 0.10324922887707333, 'num_leaves': 2820, 'max_depth': 4, 'min_data_in_leaf': 7100, 'max_bin': 272, 'lambda_l1': 45, 'lambda_l2': 15, 'min_gain_to_split': 11.044175902385101, 'bagging_fraction': 0.4, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.5.
C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\optuna

...

C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_gain_to_split is set=4.021568414970308, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.021568414970308
[LightGBM] [Warning] lambda_l1 is set=100, reg_alpha=0.0 will be ignored. Current value: lambda_l1=100
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] lambda_l2 is set=25, reg_lambda=0.0 will be ignored. Current value: lambda_l2=25
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[1]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[2]	valid_0's binary_logloss: 0.573215	valid_0's auc: 0.720316
[3]	valid_0's binary_logloss: 0.564391	valid_0's 

C:\Users\Andrii\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's binary_logloss: 0.583906	valid_0's auc: 0.5
[2]	valid_0's binary_logloss: 0.573206	valid_0's auc: 0.652457
[3]	valid_0's binary_logloss: 0.565216	valid_0's auc: 0.756814
[4]	valid_0's binary_logloss: 0.556673	valid_0's auc: 0.794266
[5]	valid_0's binary_logloss: 0.547216	valid_0's auc: 0.834416
[6]	valid_0's binary_logloss: 0.538926	valid_0's auc: 0.840181
[7]	valid_0's binary_logloss: 0.529484	valid_0's auc: 0.842462
[8]	valid_0's binary_logloss: 0.522006	valid_0's auc: 0.847085
[9]	valid_0's binary_logloss: 0.516076	valid_0's auc: 0.86087
[10]	valid_0's binary_logloss: 0.510824	valid_0's auc: 0.860569
[11]	valid_0's binary_logloss: 0.505538	valid_0's auc: 0.862728
[12]	valid_0's binary_logloss: 0.500565	valid_0's auc: 0.866098
[13]	valid_0's binary_logloss: 0.494813	valid_0's auc: 0.872752
[14]	valid_0's binary_logloss: 0.489039	valid_0's auc: 0.878469
[15]	valid_0's binary_logloss: 0.489039	valid_0's auc: 0.878469
[16]	valid_0's binary_logloss: 0.484373	valid_0's auc: 

KeyboardInterrupt: 

In [12]:
print(f"\tBest value (roc auc): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (roc auc): 0.96418
	Best params:
		n_estimators: 10000
		learning_rate: 0.20390724897401358
		num_leaves: 2920
		max_depth: 6
		min_data_in_leaf: 300
		max_bin: 237
		lambda_l1: 0
		lambda_l2: 0
		min_gain_to_split: 9.790858174571884
		bagging_fraction: 0.9
		bagging_freq: 1
		feature_fraction: 0.9


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# best_params = {
#         "n_estimators": 10000,
#         "learning_rate": 0.2529083611468759,
#         "num_leaves": 2760,
#         "max_depth": 9,
#         "min_data_in_leaf": 500,
#         "max_bin": 219,
#         "lambda_l1": 0,
#         "lambda_l2": 75  ,
#         "min_gain_to_split": 0.8789919953862727,
#         "bagging_fraction": 0.9,
#         "bagging_freq": 1,
#         "feature_fraction": 0.7,
#     }

model = lgb.LGBMClassifier(objective="binary", **study.best_params)
model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    early_stopping_rounds=100
    # callbacks=[
    #     LightGBMPruningCallback(study.best_trial, "auc")
    # ],
)
y_pred_val = model.predict_proba(X_val)[:, 1]
print(roc_auc_score(y_val, y_pred_val))
y_pred_train = model.predict_proba(X_train)[:, 1]
print(roc_auc_score(y_train, y_pred_train))

[1]	valid_0's binary_logloss: 0.437283	valid_0's auc: 0.849728
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.42843	valid_0's auc: 0.881053
[3]	valid_0's binary_logloss: 0.419539	valid_0's auc: 0.893296
[4]	valid_0's binary_logloss: 0.410678	valid_0's auc: 0.912534
[5]	valid_0's binary_logloss: 0.40375	valid_0's auc: 0.909621
[6]	valid_0's binary_logloss: 0.395146	valid_0's auc: 0.920147
[7]	valid_0's binary_logloss: 0.387635	valid_0's auc: 0.925043
[8]	valid_0's binary_logloss: 0.380974	valid_0's auc: 0.925421
[9]	valid_0's binary_logloss: 0.37465	valid_0's auc: 0.927225
[10]	valid_0's binary_logloss: 0.368122	valid_0's auc: 0.931208
[11]	valid_0's binary_logloss: 0.361202	valid_0's auc: 0.936803
[12]	valid_0's binary_logloss: 0.355924	valid_0's auc: 0.937103
[13]	valid_0's binary_logloss: 0.350263	valid_0's auc: 0.938672
[14]	valid_0's binary_logloss: 0.345014	valid_0's auc: 0.940643
[15]	valid_0's binary_logloss: 0.33974	valid_0's auc:

In [ ]:
y_pred_test = model.predict_proba(X_test)[:, 1]

In [ ]:
y_pred_test

array([2.78075367e-04, 3.33125022e-03, 9.41735169e-01, ...,
       6.79837923e-04, 5.31364850e-04, 5.48373890e-02])

In [ ]:
res = pd.DataFrame(data={'Predicted': y_pred_test}, index=X_test.index)

In [ ]:
res

,Predicted
Id,
6107424960806386948,0.000278
-1808613790090531322,0.003331
-2649749947427248381,0.941735
4684257435812939224,0.390402
-7030158396775705152,0.048197
...,...
-2211304808281344572,0.000014
6416554621804862174,0.019081
-2318403417965670905,0.000680


In [ ]:
res.to_csv('data/submission_lgb_3.csv')